In [1]:
import pandas as pd
import re
import os
import time

C:\Users\Osyi Cozy\AppData\Local\Temp\ipykernel_72016\2340826034.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
filepath = "./results/pemilu2024-2024-03-06_11-24-21.csv"
df = pd.read_csv(filepath)
df

,text,author,reply,retweet,like,date
0,Feri Amsari I Aktor Terbesar Kecurangan Pemilu...,Mahkamah Keluarga,0,1,2,2024-03-06T01:19:05.000Z
1,Subscribe to Premium+ to go ad-free in For You.,Premium,147,80,1.5K,NaN
2,Masyarakat diharapkan untuk bersabar dan menah...,Titah Pandita,0,0,0,2024-03-06T02:38:13.000Z
3,Jam 4 KJMU MKMK Pagi-pagi Semalem Pemilu 2024 ...,Puspa Yuniza,0,0,0,2024-03-06T02:50:02.000Z
4,Patroli dialogis yang humanis unit Samapta Pol...,Polsek Pancoran,0,0,0,2024-03-06T02:39:01.000Z
...,...,...,...,...,...,...
485,"Pemilu 2024 Sangat Buruk, Ahmad Rofiq: Partai ...",Partai Perindo,0,0,2,2024-03-05T17:29:22.000Z
486,TVR 120- Komisi III DPR RI : Gakkumdu Pemilu D...,tvr.parlemen,0,0,1,2024-03-05T12:35:22.000Z
487,"Antisipasi Guantibmas Pasca\nPemilu 2024, Polr...",Makhin Susanto,0,0,0,2024-03-05T13:08:08.000Z
488,Proses pergeseran logistik Pemilu 2024 merupak...,Nidyna10,3,12,15,2024-03-05T23:25:14.000Z


In [3]:
def save_data(filename, df):
    filepath = f"./results/{filename}"
    df.to_csv(filepath, index=False)
    print('file saved!')

# Pre-processing Data

## Remove Null/Nan data

In [4]:
display(f"sebelum:{df.shape}")
df.dropna(inplace=True)
display(f"sesudah:{df.shape}")

'sebelum:(490, 6)'

'sesudah:(447, 6)'

## Cleaning Date
- temukan date yg tidak berbentuk time
- temukan start date dan end date

In [5]:
display(f"start date: {df['date'].min()}")
display(f"end date: {df['date'].max()}")

'start date: 2024-03-05T11:05:36.000Z'

'end date: 2024-03-06T03:24:10.000Z'

## Cleaning Text
- remove duplicate by text
- remove hastag ,mention, emoticon / non-ascii, etc.
- cari kata tidak baku
- word cloud

##### 1. Remove duplicate by text

*ada 218 duplicate data berdasarkan text*

In [6]:
df_duplicate = df[df.duplicated(subset='text', keep=False)]
df_duplicate.shape

(58, 6)

In [7]:
display(f"sebelum:{df.shape}")
df.drop_duplicates(subset=['text'], keep=False, inplace=True)
display(f"sesudah:{df.shape}")

'sebelum:(447, 6)'

'sesudah:(389, 6)'

In [8]:
df.reset_index(drop=True, inplace=True)

##### 2. Remove hastag ,mention, emoticon / non-ascii, dll

In [13]:
def clean_text(text):
    # Menghapus hashtag
    text = re.sub(r'#\w+', ' ', text)
    # Menghapus mention
    text = re.sub(r'@\w+', ' ', text)
    # Menghapus mention
    text = text.replace('\n', ' ')
    # Menghapus tautan (link)
    text = re.sub(r'http\S+', '', text)
    # Menghapus emoticon
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251\U0001F004]', ' ', text)
    # Menghapus karakter non-ASCII
    text = ''.join(char for char in text if ord(char) < 128)
    
    return text

In [14]:
clean_text('asdasd #pemilu2024')

'asdasd #pemilu2024'

In [10]:
df['cleaned_text'] = df['text'].apply(clean_text)
df[['text', 'cleaned_text']]

,text,cleaned_text
0,Ingat jangan golput ayok datang ke TPS.\n\n#Ay...,Ingat jangan golput ayok datang ke TPS.
1,Satu suara sangat berharga sekali demi persatu...,Satu suara sangat berharga sekali demi persatu...
2,Siap... Untuk menyukseskan pesta demokrasi.\n\...,Siap... Untuk menyukseskan pesta demokrasi.
3,"PKS, partai yang memberikan suara dan menjadi ...","PKS, partai yang memberikan suara dan menjadi ..."
4,"PKS, partai yang memperjuangkan nilai-nilai Pa...","PKS, partai yang memperjuangkan nilai-nilai Pa..."
...,...,...
1052,ayooo simak apa aja nih yang perlu dibawa saat...,ayooo simak apa aja nih yang perlu dibawa saat...
1053,"ges #AyoSiapSiapNyoblos ya, jangan lupa tangga...","ges ya, jangan lupa tanggal 14 februari, jng..."
1054,"Yuk aaah kalaupun bingung, usahakan jangan gol...","Yuk aaah kalaupun bingung, usahakan jangan gol..."
1055,hellow pasti kalian sudah punya bayangan kan m...,hellow pasti kalian sudah punya bayangan kan m...


##### 3. Cari kata tidak baku
mecari kata tidak baku menggunakan pendekatan:
1. memanfaatkan google translate library untuk cek apakah kata itu baku atau tidak
2. menggunakan wordnet per 50 data untuk melihat setiap kata untuk mengumpulkan kata2 tidak baku

In [14]:
if not is_indonesian_word('bukti&bukti'):
    print('tidak baku')
else:
    print('baku')

bukti bukti bukti&bukti
tidak baku


In [13]:
from googletrans import Translator
translator = Translator()

start_, end_ = None, None

# Fungsi untuk memeriksa apakah sebuah kata adalah kata baku dalam Bahasa Indonesia
def is_indonesian_word(word):
    time.sleep(0.5)
    # Lakukan terjemahan dari Bahasa Indonesia ke Bahasa Indonesia
    translated_word = translator.translate(word, src='id', dest='id').text
    print(translated_word.lower(), word.lower())
    # Jika hasil terjemahan tidak berubah, maka kata tersebut adalah kata baku
    if translated_word.lower() == word.lower():
        return True
    else:
        return False

# Fungsi untuk memeriksa kata-kata tidak baku dalam teks dan membuat mapping
def map_non_baku_words(row):
    global df, start_

    start_ = row.name

    non_baku_words = []
    words = row['cleaned_text'].split()
    # Memeriksa setiap kata dalam teks
    for word in words:
        # Jika kata tidak baku, tambahkan ke daftar kata tidak baku
        if not is_indonesian_word(word):
            non_baku_words.append(word)
    # Jika ada kata tidak baku, kembalikan daftar kata-kata tidak baku sebagai string yang dipisahkan oleh koma
    # dan indeks baris DataFrame
    # Jika tidak ada kata tidak baku, kembalikan None
    
    if non_baku_words:
        res = ', '.join(non_baku_words)
        print(f"{row.name} ({len(non_baku_words)}): {res}")
        df.loc[row.name, 'non_baku'] = res
        save_data('AyoSiapSiapNyoblos-fixed.csv', df=df)
        return res 
    else:
        print(f"{row.name} ({len(non_baku_words)})")
        df.loc[row.name, 'non_baku'] = None
        save_data('AyoSiapSiapNyoblos-fixed.csv', df=df)
        return None


In [6]:
def checkWords(start, end):
    global start_, end_

    
    if start_ == None:
        start_ = start
    end_ = end
    
    print(f"start from:{start_} - {end_}")

    try:
        df.iloc[start_:end_].apply(map_non_baku_words, axis=1)
    except Exception as e:
        print(f"Error: {e}")
        time.sleep(60)
        checkWords(start_, end_)



checkWords(683, df.shape[0])

start from:683 - 1057
683 (0)
file saved!
684 (0)
file saved!
685 (0)
file saved!
686 (0)
file saved!
687 (0)
file saved!
688 (0)
file saved!
689 (0)
file saved!
690 (0)
file saved!
691 (0)
file saved!
692 (0)
file saved!
693 (0)
file saved!
694 (0)
file saved!
695 (0)
file saved!
696 (0)
file saved!
697 (0)
file saved!
698 (0)
file saved!
699 (0)
file saved!
700 (0)
file saved!
701 (0)
file saved!
702 (0)
file saved!
703 (0)
file saved!
704 (0)
file saved!
705 (0)
file saved!
706 (0)
file saved!
707 (0)
file saved!
708 (0)
file saved!
709 (0)
file saved!
710 (0)
file saved!
711 (0)
file saved!
712 (0)
file saved!
713 (0)
file saved!
714 (0)
file saved!
715 (0)
file saved!
716 (0)
file saved!
717 (0)
file saved!
718 (0)
file saved!
719 (0)
file saved!
720 (0)
file saved!
721 (0)
file saved!
722 (0)
file saved!
723 (0)
file saved!
724 (0)
file saved!
725 (0)
file saved!
726 (0)
file saved!
727 (0)
file saved!
728 (0)
file saved!
729 (1): dgn
file saved!
730 (0)
file saved!
731 (0)
file 

In [13]:
df

,text,author,reply,retweet,like,date,cleaned_text
0,Ingat jangan golput ayok datang ke TPS.\n\n#Ay...,N E Y A V A,0,0.0,0.0,2024-02-12T13:48:28.000Z,Ingat jangan golput ayok datang ke TPS.
1,Satu suara sangat berharga sekali demi persatu...,Yuli Endah,0,1.0,1.0,2024-02-12T14:22:34.000Z,Satu suara sangat berharga sekali demi persatu...
2,Siap... Untuk menyukseskan pesta demokrasi.\n\...,Yuli Endah,0,1.0,1.0,2024-02-12T13:49:19.000Z,Siap... Untuk menyukseskan pesta demokrasi.
3,"PKS, partai yang memberikan suara dan menjadi ...",#Seger8ener #AMINajadulu,0,0.0,0.0,2024-02-12T13:49:08.000Z,"PKS, partai yang memberikan suara dan menjadi ..."
4,"PKS, partai yang memperjuangkan nilai-nilai Pa...",Adhy Suryo W,0,0.0,0.0,2024-02-12T15:27:26.000Z,"PKS, partai yang memperjuangkan nilai-nilai Pa..."
...,...,...,...,...,...,...,...
1052,ayooo simak apa aja nih yang perlu dibawa saat...,caa,0,0.0,0.0,2024-02-12T10:02:26.000Z,ayooo simak apa aja nih yang perlu dibawa saat...
1053,"ges #AyoSiapSiapNyoblos ya, jangan lupa tangga...",diraaa,0,0.0,0.0,2024-02-12T10:05:23.000Z,"ges ya, jangan lupa tanggal 14 februari, jng..."
1054,"Yuk aaah kalaupun bingung, usahakan jangan gol...",Sansan84,0,0.0,0.0,2024-02-12T10:09:19.000Z,"Yuk aaah kalaupun bingung, usahakan jangan gol..."
1055,hellow pasti kalian sudah punya bayangan kan m...,Relie,0,0.0,0.0,2024-02-12T10:03:43.000Z,hellow pasti kalian sudah punya bayangan kan m...


In [16]:
df.loc[900]

2696

In [78]:
save_data('pemilu2024-fixed.csv', df=df)

file saved!
